In [1]:
import torch
import transformers
import torch.nn as nn
import math

In [2]:
hidden_states = torch.randn(2, 3, 4)
q_proj = nn.Linear(4, 4, bias=False)
k_proj = nn.Linear(4, 4, bias=False)
v_proj = nn.Linear(4, 4, bias=False)
o_proj = nn.Linear(4, 4, bias=False)

In [3]:
q = q_proj(hidden_states)
k = k_proj(hidden_states)
v = v_proj(hidden_states)

In [4]:
mask = torch.ones(2, 3)
print(mask)
mask[1][2] = 0
print(mask)

tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., 1., 1.],
        [1., 1., 0.]])


In [5]:
P = q.matmul(k.transpose(-2, -1)) / math.sqrt(4)
print(P.shape)
print(P)


causal_mask = (torch.triu(torch.ones(2, 3, 3)) == 1).transpose(1, 2)
print(causal_mask.shape)
P = P * causal_mask
print(P)

torch.Size([2, 3, 3])
tensor([[[-1.7162, -1.4536, -0.4920],
         [-0.9353, -0.8119, -0.2695],
         [ 0.0611,  0.0057,  0.0570]],

        [[-0.3247,  0.1361, -0.5795],
         [ 0.6162, -0.1619,  0.7786],
         [-0.2297,  0.4198, -0.2843]]], grad_fn=<DivBackward0>)
torch.Size([2, 3, 3])
tensor([[[-1.7162, -0.0000, -0.0000],
         [-0.9353, -0.8119, -0.0000],
         [ 0.0611,  0.0057,  0.0570]],

        [[-0.3247,  0.0000, -0.0000],
         [ 0.6162, -0.1619,  0.0000],
         [-0.2297,  0.4198, -0.2843]]], grad_fn=<MulBackward0>)


In [6]:
A = nn.functional.softmax(P, dim=-1, dtype=torch.float32).to(q.dtype)
O = torch.matmul(A, v)



In [7]:
q_heads = q.view(2, 3, 2, 2).transpose(1, 2)
k_heads = k.view(2, 3, 2, 2).transpose(1, 2)
v_heads = v.view(2, 3, 2, 2).transpose(1, 2)
print(q_heads.shape,k_heads.shape,v_heads.shape)


torch.Size([2, 2, 3, 2]) torch.Size([2, 2, 3, 2]) torch.Size([2, 2, 3, 2])


In [8]:
P = q_heads.matmul(k_heads.transpose(-2, -1)) / math.sqrt(4)
print(P.shape)
print(P)


causal_mask = (torch.triu(torch.ones(2, 3, 3)) == 1).transpose(1, 2)
print(causal_mask.shape)
P = P * causal_mask
print(P)

A = nn.functional.softmax(P, dim=-1, dtype=torch.float32).to(q.dtype)
O = torch.matmul(A, v_heads)

O = O.transpose(1, 2).contiguous() #下面如果用reshape，可以不用contiguous

O = O.reshape(2, 3, -1)
print(O.shape)

output = o_proj(O)


print(output.shape)

torch.Size([2, 2, 3, 3])
tensor([[[[-1.6499e+00, -1.2629e+00, -3.9503e-01],
          [-8.6208e-01, -6.3481e-01, -1.7792e-01],
          [-5.7256e-02,  4.2234e-03,  4.0933e-02]],

         [[-6.6269e-02, -1.9071e-01, -9.6926e-02],
          [-7.3249e-02, -1.7711e-01, -9.1537e-02],
          [ 1.1836e-01,  1.4750e-03,  1.6079e-02]]],


        [[[-3.4250e-01,  2.4986e-01, -5.2080e-01],
          [ 3.2194e-01, -1.0455e-01,  6.4518e-01],
          [-4.1561e-01,  5.9331e-01, -2.8545e-01]],

         [[ 1.7789e-02, -1.1378e-01, -5.8736e-02],
          [ 2.9427e-01, -5.7370e-02,  1.3344e-01],
          [ 1.8590e-01, -1.7352e-01,  1.1639e-03]]]], grad_fn=<DivBackward0>)
torch.Size([2, 3, 3])
tensor([[[[-1.6499e+00, -0.0000e+00, -0.0000e+00],
          [-8.6208e-01, -6.3481e-01, -0.0000e+00],
          [-5.7256e-02,  4.2234e-03,  4.0933e-02]],

         [[-6.6269e-02, -0.0000e+00, -0.0000e+00],
          [-7.3249e-02, -1.7711e-01, -0.0000e+00],
          [ 1.1836e-01,  1.4750e-03,  1.6079e-02]